# Pre-canned Queries for Python
## Function to call Financial Filing 2
Query for Filing Document and Organization Reference Data for Thomson Reuters.


tags :     `Filings`, `Documents`, `Summary`
## Import the library and load credentials
Credentials used by this notebook are stored in the config_files/credentials.ipynb. Please edit credentials.ipynb to set your credentials and run the next cell to continue with the scripts 

In [ ]:
%run ../../../config_files/credentials.ipynb 
%run ../../../config_files/environment_selection.ipynb

To connect to the Refinitiv Data and opening a session,given function is used. To find out more about the library,you can visit [refinitiv-data library](https://pypi.org/project/refinitiv-data/)

In [ ]:
session = rd.session.platform.Definition(
    app_key=APP_KEY,
    grant=rd.session.platform.GrantPassword(
        username=RD_LOGIN, password=RD_PASSWORD
    ),
).get_session()
session.open()


## Graph call
Graph call with given sets of samples below. 

In [ ]:
%%writefile query_cache
query FinancialFiling($OrganizationId: Long, $FilingDateFrom: DateTime!, $FilingDateTo: DateTime!, $LanguageId: Long) {
  FinancialFiling(filter: {AND: [{FilingDocument: {Identifiers: {OrganizationId: {EQ: $OrganizationId}}}}, {FilingDocument: {DocumentSummary: {FilingDate: {BETWN: {FROM: $FilingDateFrom, TO: $FilingDateTo}}}}}]}, sort: {FilingDocument: {DocumentSummary: {FilingDate: DESC}}}, limit: 10) {
    FilingOrganization {
      Names {
        Name {
          OrganizationName(filter: {AND: [{LanguageId: {EQ: $LanguageId}}, {NameTypeCode: {EQ: "LNG"}}]}) {
            Name
          }
        }
      }
      Details {
        OrganizationDetails {
          JurisdictionOfIncorporation
        }
      }
    }
    FilingDocument {
      _metadata {
        totalCount
      }
      Identifiers {
        Dcn
      }
      DocId
      FinancialFilingId
      DocumentSummary {
        DocumentTitle
        FilingDate
        DocumentType
        SubmissionTypeDescription {
          Value
        }
        HighLevelCategory
        MidLevelCategory
        FeedName
      }
      FilesMetaData {
        FileName
        MimeType
      }
    }
  }
}


## Sample :  Find by Organization ID, dates, and language

In [ ]:

variables = {'OrganizationId': '4295861160', 'LanguageId': '505062', 'FilingDateFrom': '2020-01-01T00:00:00Z', 'FilingDateTo': '2020-12-31T00:00:00Z'}

In [3]:
output_format = 'json'  # available: json
%run ../../../config_files/config_request_handler.ipynb

{
    "data": {
        "FinancialFiling": [
            {
                "FilingOrganization": {
                    "Names": {
                        "Name": {
                            "OrganizationName": [
                                {
                                    "Name": "THOMSON REUTERS CORPORATION"
                                }
                            ]
                        }
                    },
                    "Details": {
                        "OrganizationDetails": {
                            "JurisdictionOfIncorporation": "ONTARIO"
                        }
                    }
                },
                "FilingDocument": {
                    "_metadata": {
                        "totalCount": null
                    },
                    "Identifiers": [
                        {
                            "Dcn": "sd05451593"
                        }
                    ],
                    "DocId": "56357098",
         

## Close the default session when done

In [ ]:
session.close()